<a href="https://colab.research.google.com/github/Junhojuno/keras-tutorial/blob/master/07_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 함수형 API
- 기존 Sequential과 함수형 API 비교
- compile부분부터는 기존 Sequential과 동일

In [0]:
from keras.models import Sequential, Model
from keras.layers import *
from keras import Input

# 기존 Sequential 모델
seq_model = Sequential()
seq_model.add(Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(Dense(32, activation='relu'))
seq_model.add(Dense(10, activation='softmax'))

seq_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 함수형 API
input_tensor = Input(shape=(64,))
x = Dense(32, activation='relu')(input_tensor)
x = Dense(32, activation='relu')(x)
output_tensor = Dense(10, activation='softmax')(x)

functional_model = Model(input_tensor, output_tensor)

functional_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


### 다중 입력 모델
- 함수 API는 다중입력모델을 만드는데 사용할 수 있다.
- 가장 간단한 Question-Answering 모델 구현
  - 입력으로 정보가 담긴 text와 질문이 들어감
  - 답을 출력하도록 하는 모델 구성

In [2]:
from keras.models import Model
from keras.layers import *
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = Embedding(input_dim=text_vocabulary_size, output_dim=64)(text_input)
encoded_text = LSTM(32)(embedded_text) # (sample,features)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = Embedding(input_dim=question_vocabulary_size, output_dim=32)(question_input)
encoded_question = LSTM(16)(embedded_question) # (sample,features)

# encoding된 질문과 텍스트를 연결
concatenated = concatenate([encoded_text, encoded_question], axis=-1)

answer = Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)


model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
_____________________________________

```python
# 여러 입력데이터를 어떻게 집어넣을 것인가?
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answer = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answer = to_categorical(answer) # one-hot encoding

# 입력방식 첫번째 : numpy로 만들어 넣어버리기
model.fit(x=[text, question], y=answer, epochs=10, batch_size=128)

# 입력방식 두번째 : numpy로 만들고 dictionary형태로 만들어 넣기
model.fit(x={'text':text, 'question':question}, answer, epochs=10, batch_size=128)
```

### 다중 출력모델 (multi-output / multi-head)
- 예를 들면, 포스팅을 입력으로 받아 작성자의 나이, 성별, 소득수준 등을 예측하는 방식

In [6]:
from keras.layers import *
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = Embedding(input_dim=vocabulary_size, output_dim=256)(posts_input)
x = Conv1D(filters=128, kernel_size=5, activation='relu')(embedded_posts)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)

# 출력값마다 다르게 loss function을 지정해주자
age_prediction = Dense(1, name='age')(x)
sex_prediction = Dense(1, activation='sigmoid', name='sex')(x)
income_prediction = Dense(num_income_groups, activation='softmax', name='income')(x)

model = Model(posts_input, [age_prediction, sex_prediction, income_prediction])

# 위에서 name을 지정해줬기때문에 dictionary로 작성가능
# 지정하지 않았다면, 그냥 순서대로 list로 구성하면 됨.
model.compile(optimizer='rmsprop', loss={'age':'mse', 'sex': 'binary_crossentropy', 'income':'categorical_crossentropy'})

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, None, 128)    163968      embedding_6[0][0]                
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, None, 128)    0           conv1d_16[0][0]                  
__________________________________________________________________________________________________
conv1d_17 

##### 하지만, 이렇게 했을때 문제가 있다.
  - 전체 loss가 각 loss의 합으로 계산되고 이를 최소화하려는 방향으로 최적화가 진행된다.
  - 이렇게 되면 loss가 큰 문제의 loss를 줄이기 위해 한쪽으로 최적화가 치우칠수 있다.
  - 이를 방지하기위해 compile에 loss_weights를 줄 수 있다.(아래 코드와 같이)
  - 가중치 부여기준은 mse가 대략 3~5가 나오고, 이진분류는 대략 0.1정도 나오기 때문에 비슷하게 맞춰준다.

```python
model.compile(optimizer='rmsprop', loss={'age' : 'mse', 'sex' :  'binary_crossentropy', 'income':'categorical_crossentropy'}, loss_weights={'age' : 0.25, 'sex' : 10., 'income' : 1.})

```

##### model fit하는 방식은 비슷하다

```python
model.fit(posts, [age_targets, sex_targets, income_targets], epochs=10, batch_size=64)
```